In [1]:
from ray.rllib.algorithms.ppo import PPO

from esc.epanet_env import N_SIMULATION_STEPS, EPANETEnv

N_TRAINING_ITERATIONS = 1000
# Configure the algorithm.
config = {
    "env_config": {},
    # Use 5 environment workers (aka "rollout workers") that parallelly
    # collect samples from their own environment clone(s).
    "num_workers": 9,
    "horizon": N_SIMULATION_STEPS,
    # Change this to "framework: torch", if you are using PyTorch.
    # Also, use "framework: tf2" for tf2.x eager execution.
    "framework": "torch",
    # Tweak the default model provided automatically by RLlib,
    # given the environment's observation- and action spaces.
    "model": {
        "fcnet_hiddens": [256, 256],
        "fcnet_activation": "relu",
    },
    "batch_mode": "complete_episodes",
    "simple_optimizer": True,
    "recreate_failed_workers": True,
    "log_sys_usage": False,
    # Set up a separate evaluation worker set for the
    # `algo.evaluate()` call after training (see below).
    "evaluation_num_workers": 1,
    # Only for evaluation runs, render the env.
    # "evaluation_config": {
    #     "render_env": True,
    # },
    "log_level": "ERROR"
}

# Create our RLlib Trainer.
algo = PPO(env=EPANETEnv, config=config)
# algo = PPO.from_checkpoint("/Users/maxdumas/ray_results/PPO_EPANETEnv_2022-11-27_22-44-29tztzkh8w/checkpoint_000451")

# Run it for n training iterations. A training iteration includes
# parallel sample collection by the environment workers as well as
# loss calculation on the collected batch and a model update.
for i in range(N_TRAINING_ITERATIONS):
    print(algo.train())

    if i % 50 == 0:
        checkpoint = algo.save()
        print(f"Checkpoint saved to {checkpoint}")

# Evaluate the trained Trainer (and render each timestep to the shell's
# output).
algo.evaluate()

2022-12-01 11:44:33,390	WARNING deprecation.py:47 -- DeprecationWarning: `simple_optimizer` has been deprecated. This will raise an error in the future!
2022-12-01 11:44:33,395	WARNING algorithm.py:2531 -- You have specified 1 evaluation workers, but your `evaluation_interval` is None! Therefore, evaluation will not occur automatically with each call to `Algorithm.train()`. Instead, you will have to call `Algorithm.evaluate()` manually in order to trigger an evaluation run.
2022-12-01 11:44:33,396	WARNING ppo.py:351 -- `train_batch_size` (4000) cannot be achieved with your other settings (num_workers=9 num_envs_per_worker=1 rollout_fragment_length=200)! Auto-adjusting `rollout_fragment_length` to 445.
2022-12-01 11:44:35,330	INFO worker.py:1528 -- Started a local Ray instance.


(RolloutWorker pid=35626) EPANET version 20200 loaded (EPyT version 1.0.1).
(RolloutWorker pid=35626) Input File BUILDING.inp loaded successfully.
(RolloutWorker pid=35626) 
(RolloutWorker pid=35628) EPANET version 20200 loaded (EPyT version 1.0.1).
(RolloutWorker pid=35628) Input File BUILDING.inp loaded successfully.
(RolloutWorker pid=35628) 
(RolloutWorker pid=35624) EPANET version 20200 loaded (EPyT version 1.0.1).
(RolloutWorker pid=35624) Input File BUILDING.inp loaded successfully.
(RolloutWorker pid=35624) 
(RolloutWorker pid=35625) EPANET version 20200 loaded (EPyT version 1.0.1).
(RolloutWorker pid=35625) Input File BUILDING.inp loaded successfully.
(RolloutWorker pid=35625) 
(RolloutWorker pid=35630) EPANET version 20200 loaded (EPyT version 1.0.1).
(RolloutWorker pid=35630) Input File BUILDING.inp loaded successfully.
(RolloutWorker pid=35630) 
(RolloutWorker pid=35627) EPANET version 20200 loaded (EPyT version 1.0.1).
(RolloutWorker pid=35627) Input File BUILDING.inp load

2022-12-01 11:44:42,776	WARNING util.py:66 -- Install gputil for GPU system monitoring.


(RolloutWorker pid=35679) EPANET version 20200 loaded (EPyT version 1.0.1).
(RolloutWorker pid=35679) Input File BUILDING.inp loaded successfully.
(RolloutWorker pid=35679) 
(RolloutWorker pid=35626) EPANET version 20200 loaded (EPyT version 1.0.1).
(RolloutWorker pid=35626) Input File BUILDING.inp loaded successfully.
(RolloutWorker pid=35626) 
(RolloutWorker pid=35628) EPANET version 20200 loaded (EPyT version 1.0.1).
(RolloutWorker pid=35628) Input File BUILDING.inp loaded successfully.
(RolloutWorker pid=35628) 
(RolloutWorker pid=35624) EPANET version 20200 loaded (EPyT version 1.0.1).
(RolloutWorker pid=35624) Input File BUILDING.inp loaded successfully.
(RolloutWorker pid=35624) 
(RolloutWorker pid=35629) EPANET version 20200 loaded (EPyT version 1.0.1).
(RolloutWorker pid=35629) Input File BUILDING.inp loaded successfully.
(RolloutWorker pid=35629) 
(RolloutWorker pid=35625) EPANET version 20200 loaded (EPyT version 1.0.1).
(RolloutWorker pid=35625) Input File BUILDING.inp load

NaN or Inf found in input tensor.
NaN or Inf found in input tensor.


{'custom_metrics': {}, 'episode_media': {}, 'num_recreated_workers': 0, 'info': {'learner': {'default_policy': {'learner_stats': {'allreduce_latency': 0.0, 'grad_gnorm': 13.917009531147778, 'cur_kl_coeff': 0.0019775390624999995, 'cur_lr': 5.0000000000000016e-05, 'total_loss': inf, 'policy_loss': -0.003041862801183015, 'vf_loss': 1.9174910270919403, 'vf_explained_var': -1.0, 'kl': inf, 'entropy': 0.12476672268627832, 'entropy_coeff': 0.0}, 'model': {}, 'custom_metrics': {}, 'num_agent_steps_trained': 125.15625}}, 'num_env_steps_sampled': 68085, 'num_env_steps_trained': 68085, 'num_agent_steps_sampled': 68085, 'num_agent_steps_trained': 68085}, 'sampler_results': {'episode_reward_max': 6.656611923496424, 'episode_reward_min': 3.6585912860005414, 'episode_reward_mean': 4.902779757479964, 'episode_len_mean': 2880.0, 'episode_media': {}, 'episodes_this_iter': 0, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean': {}, 'custom_metrics': {}, 'hist_stats': {'episode_reward':

NaN or Inf found in input tensor.
NaN or Inf found in input tensor.


{'custom_metrics': {}, 'episode_media': {}, 'num_recreated_workers': 0, 'info': {'learner': {'default_policy': {'learner_stats': {'allreduce_latency': 0.0, 'grad_gnorm': 120.11593419412772, 'cur_kl_coeff': 0.00013904571533203123, 'cur_lr': 5.0000000000000016e-05, 'total_loss': inf, 'policy_loss': 0.001161741220857948, 'vf_loss': 4.829155081262191, 'vf_explained_var': 0.6603338334088524, 'kl': inf, 'entropy': 3.570833809732326e-15, 'entropy_coeff': 0.0}, 'model': {}, 'custom_metrics': {}, 'num_agent_steps_trained': 125.15625}}, 'num_env_steps_sampled': 96120, 'num_env_steps_trained': 96120, 'num_agent_steps_sampled': 96120, 'num_agent_steps_trained': 96120}, 'sampler_results': {'episode_reward_max': 153.94859155961382, 'episode_reward_min': 3.6585912860005414, 'episode_reward_mean': 21.351905655823696, 'episode_len_mean': 2880.0, 'episode_media': {}, 'episodes_this_iter': 0, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean': {}, 'custom_metrics': {}, 'hist_stats': {

/opt/homebrew/Caskroom/mambaforge/base/envs/engineering-smart-cities/lib/python3.10/site-packages/ray/rllib/algorithms/algorithm.py:1688: DeprecationWarning: invalid escape sequence '\|'
  """Adds a new policy to this Algorithm.


ValueError: Could not find key 'grad_gnorm' in some `input_trees`. Please ensure the structure of all `input_trees` are compatible with `shallow_tree`. The last valid path yielded was ('learner_stats', 'entropy_coeff').